In [36]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import sys
import os
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [28]:
writer = SummaryWriter('./runs/classification_all/')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [29]:
class COMDataset(Dataset):
    def __init__(self, production_dir, transform=None):
        self.transform = transform
        self.prodiction_dir = production_dir
        self.com_dir = os.path.join(production_dir, 'com_dir')
        self.com_list = []
        self.com_files = os.listdir(self.com_dir)
        self.com_files = [f for f in os.listdir(self.com_dir) if f.endswith('.txt')]
        self.com_files.sort(key=self.sort_coms)
        for window, file in enumerate(self.com_files):
            self.com_list.append(pd.read_csv(os.path.join(self.com_dir, file), header=None, names=[window], usecols=[0]))
        self.com_list = np.array(self.com_list)
        for idxs in range(len(self.com_list)):
            for i in range(len(self.com_list[idxs])):
                self.com_list[idxs][i][0] = np.float64(self.com_list[idxs][i][0][9:].strip()) 
        self.com_list = self.com_list.squeeze()
        
        self.n_windows = len(self.com_files)
        self.window_paths = [os.path.join(self.prodiction_dir, str(window), 'hb_observable.txt') for window in range(self.n_windows)]
        self.hb_list = []
        for window, path in enumerate(self.window_paths):
            self.hb_list.append(pd.read_csv(path, header=None,names=[window], usecols=[0]))        
        self.hb_list = np.array(self.hb_list).squeeze()
        
        self.x = np.reshape(self.com_list, self.com_list.shape[0] * self.com_list.shape[1])
        self.y = np.reshape(self.hb_list, self.hb_list.shape[0] * self.hb_list.shape[1])

        self.n_samples = self.x.shape[0]
        
    def sort_coms(self, file):
        # Method to sort com files by window number
        var = int(file.split('_')[-1].split('.')[0])
        return var
    
    def __getitem__(self, index):
        sample = self.x[index], self.y[index]
        if self.transform:
            sample = self.transform(sample)
        return sample
    
    def __len__(self):
        return self.n_samples

In [81]:
production_dir = '/scratch/mlsample/ipy_oxDNA/ipy_oxdna_examples/duplex_melting/us_melting/production/'
dataset  = COMDataset(production_dir)
batch_size = 4096
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

In [82]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

In [83]:
#hyper params
input_size = 1 # 28x28
hidden_size = 64
num_classes = 10
num_epochs = 10
learning_rate = 0.0001

In [84]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [85]:
running_loss = 0.0
running_correct = 0
#trainin loop
n_total_steps = len(train_loader)
for epoch in tqdm(range(num_epochs)):
    for i, (x, y) in enumerate(train_loader):
        x = torch.tensor(x, dtype=torch.float32)
        x = x.view(x.shape[0], 1)
        x = x.to(device)
        y = y.to(device)
        #forward
        
        outputs = model(x)
        loss = criterion(outputs, y)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        running_correct += (predicted ==y).sum().item()
        
        if (i+1) % 100 == 0:
            print(f'epoch {epoch +1} / {num_epochs}, step {i + 1}/ {n_total_steps}, loss = {loss.item():.4f}')
            writer.add_scalar('training loss', running_loss / 100, epoch * n_total_steps + i)
            writer.add_scalar('accuracy', running_correct / batch_size / 100, epoch * n_total_steps + i)
            running_loss = 0.0
            running_correct = 0

  0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipykernel_753641/1036382575.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)


epoch 1 / 10, step 100/ 928, loss = 2.2507
epoch 1 / 10, step 200/ 928, loss = 1.5442
epoch 1 / 10, step 300/ 928, loss = 1.1364
epoch 1 / 10, step 400/ 928, loss = 0.9641
epoch 1 / 10, step 500/ 928, loss = 0.8054
epoch 1 / 10, step 600/ 928, loss = 0.7487
epoch 1 / 10, step 700/ 928, loss = 0.6989
epoch 1 / 10, step 800/ 928, loss = 0.6796
epoch 1 / 10, step 900/ 928, loss = 0.6250


 10%|█         | 1/10 [00:16<02:26, 16.27s/it]

epoch 2 / 10, step 100/ 928, loss = 0.5747
epoch 2 / 10, step 200/ 928, loss = 0.5531
epoch 2 / 10, step 300/ 928, loss = 0.5226
epoch 2 / 10, step 400/ 928, loss = 0.5411
epoch 2 / 10, step 500/ 928, loss = 0.5180
epoch 2 / 10, step 600/ 928, loss = 0.4837
epoch 2 / 10, step 700/ 928, loss = 0.5070
epoch 2 / 10, step 800/ 928, loss = 0.4560
epoch 2 / 10, step 900/ 928, loss = 0.4559


 20%|██        | 2/10 [00:29<01:55, 14.42s/it]

...

 80%|████████  | 8/10 [02:09<00:36, 18.06s/it]

epoch 9 / 10, step 100/ 928, loss = 0.3281
epoch 9 / 10, step 200/ 928, loss = 0.3398
epoch 9 / 10, step 300/ 928, loss = 0.3156
epoch 9 / 10, step 400/ 928, loss = 0.3266
epoch 9 / 10, step 500/ 928, loss = 0.3263
epoch 9 / 10, step 600/ 928, loss = 0.3322
epoch 9 / 10, step 700/ 928, loss = 0.3119
epoch 9 / 10, step 800/ 928, loss = 0.3082
epoch 9 / 10, step 900/ 928, loss = 0.3114


 90%|█████████ | 9/10 [02:27<00:17, 18.00s/it]

epoch 10 / 10, step 100/ 928, loss = 0.3067
epoch 10 / 10, step 200/ 928, loss = 0.3115
epoch 10 / 10, step 300/ 928, loss = 0.3196
epoch 10 / 10, step 400/ 928, loss = 0.3165
epoch 10 / 10, step 500/ 928, loss = 0.3151
epoch 10 / 10, step 600/ 928, loss = 0.3105
epoch 10 / 10, step 700/ 928, loss = 0.3082
epoch 10 / 10, step 800/ 928, loss = 0.3229
epoch 10 / 10, step 900/ 928, loss = 0.3081


100%|██████████| 10/10 [02:45<00:00, 16.55s/it]


In [87]:
with torch.no_grad():
    n_correct = 0
    n_sample = 0
    for x, y in test_loader:
        x = torch.tensor(x, dtype=torch.float32)
        x = x.view(x.shape[0], 1).to(device)
        y = y.to(device)
        outputs = model(x)
        #value, index
        _, predictions = torch.max(outputs, 1)
        n_sample += y.shape[0]
        n_correct += (predictions == y).sum().item()
        
acc = 100.0 * n_correct / n_sample
print(acc)

/tmp/ipykernel_753641/201484662.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)


87.37919978442537


In [77]:
%tensorboard --logdir './runs/classification_all/'

UsageError: Line magic function `%tensorboard` not found.
